In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [30]:
df_routes=pd.read_csv('../Files/Routes.csv')




In [31]:
df_routes.head(2)

,Route ID,Origin Port,Destination Port,Orgin Port Country,Destination Port Country,No Of Stops Between,Stops Names,"Stop Points (Orgin,[BetwwenStops],End]",Distance,Average Transit Time (days),Travel Direction,Vessel Type,No Of Travels
0,R001,Tanger Med,Rotterdam,Morocco,Netherlands,NaN,"Algeciras,Barcelona, Antwerp","[44.811855,-0.5906289]-[36.1311725,-5.4473991]...",600,18,W,Bulk carrier,3161
1,R002,Gioia Tauro,Hamburg,Italy,Germany,NaN,"Gioia Tauro,Valencia,Barcelona,Marseille,Rotte...","[38.424938, 15.899940]-[40.7127281,-74.0060152...",2468,29,E,Tank,2065


In [32]:
df_incidents=pd.read_csv('../Files/Incidents.csv')

In [33]:
df_routes.head(2)
df_incidents.head(2)

,IncidentId,RouteID,IncidentType,Severity,IncidentDate,Description
0,INC001,R538,Health Emergencies,High,5/24/23 13:58,Medical issues among crew affecting operations.
1,INC002,R401,Fuel Costs,Medium,9/25/23 13:58,Fluctuations in fuel prices affecting operatin...


In [22]:
# Convert 'date' column to datetime with the correct format
df_incidents['IncidentDate'] = pd.to_datetime(df_incidents['IncidentDate'])

C:\Users\local_jegadeesan.rameshbab12\Temp\ipykernel_35204\1789576289.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_incidents['IncidentDate'] = pd.to_datetime(df_incidents['IncidentDate'])


In [23]:
# Current date
current_date = datetime.now()

# Function to get the date 5 months ago
def date_n_months_ago(current_date, n):
    return current_date - timedelta(days=n*30)  # Approximate 5 months as 150 days

five_months_ago = date_n_months_ago(current_date, 5)

In [24]:
# Adding columns to routes dataframe
df_routes['total_incidents_count'] = 0
df_routes['high_incidents'] = 0
df_routes['high_last5Month_incidents'] = 0
df_routes['medium_incidents'] = 0
df_routes['medium_last5Month_incidents'] = 0
df_routes['low_incidents'] = 0
df_routes['low_last5Month_incidents'] = 0


In [25]:


for index,row in df_routes.iterrows():
    #total incidents and count
    route_incidents=df_incidents[df_incidents['RouteID']==row['Route ID']]   
    incidents_count=route_incidents.shape[0]
    df_routes.loc[index, 'total_incidents_count']=incidents_count
    
    #High incidents and count
    high_incidents = route_incidents[route_incidents['Severity'] == 'High']
    high_count=high_incidents.shape[0]
    high_last5Month_count=high_incidents[high_incidents['IncidentDate']>five_months_ago].shape[0]

    df_routes.loc[index, 'high_incidents']=(high_count-high_last5Month_count)
    df_routes.loc[index, 'high_last5Month_incidents']=high_last5Month_count

    
    
    #Medium incidents and count
    medium_incidents = route_incidents[route_incidents['Severity'] == 'Medium']
    medium_count=medium_incidents.shape[0]
    medium_last5Month_count=medium_incidents[medium_incidents['IncidentDate']>five_months_ago].shape[0]
    df_routes.loc[index, 'medium_incidents'] = (medium_count-medium_last5Month_count)
    df_routes.loc[index, 'medium_last5Month_incidents'] = medium_last5Month_count
    
        #Medium incidents and count
    low_incidents = route_incidents[route_incidents['Severity'] == 'Low']
    low_count=low_incidents.shape[0]
    low_last5Month_count=low_incidents[low_incidents['IncidentDate']>five_months_ago].shape[0]
    df_routes.loc[index, 'low_incidents'] = (low_count-low_last5Month_count)
    df_routes.loc[index, 'low_last5Month_incidents'] =low_last5Month_count
    


    
    


In [10]:
df_routes.head(2)

,Route ID,Origin Port,Destination Port,Orgin Port Country,Destination Port Country,No Of Stops Between,Stops Names,"Stop Points (Orgin,[BetwwenStops],End]",Distance,Average Transit Time (days),Travel Direction,Vessel Type,No Of Travels,total_incidents_count,high_incidents,high_last5Month_incidents,medium_incidents,medium_last5Month_incidents,low_incidents,low_last5Month_incidents
0,R001,Tanger Med,Rotterdam,Morocco,Netherlands,NaN,"Algeciras,Barcelona, Antwerp","[44.811855,-0.5906289]-[36.1311725,-5.4473991]...",600,18,W,Bulk carrier,3161.0,136,41,9,38,6,34,8
1,R002,Gioia Tauro,Hamburg,Italy,Germany,NaN,"Gioia Tauro,Valencia,Barcelona,Marseille,Rotte...","[38.424938, 15.899940]-[40.7127281,-74.0060152...",2468,29,E,Tank,2065.0,150,47,12,38,12,36,5


In [26]:
# Adding columns to routes dataframe
df_routes['risk_score'] = 0

In [27]:

for index,row in df_routes.iterrows():
    total_travel_count=row['No Of Travels']
    total_incidents_count=row['total_incidents_count'] #200
    high_incidents=row['high_incidents'] #*3
    high_last5Month_incidents=row['high_last5Month_incidents'] #*5
    
    medium_incidents=row['medium_incidents'] #*2
    medium_last5Month_incidents=row['medium_last5Month_incidents'] #*3
    
    low_incidents=row['low_incidents'] #*1
    low_last5Month_incidents=row['low_last5Month_incidents'] #*2
    
    risk_value=((high_incidents*5)+(high_last5Month_incidents*7)+\
        (medium_incidents*3)+(medium_last5Month_incidents*5)+\
            (low_incidents*2)+(low_last5Month_incidents*3))/total_travel_count
        
    risk_value=risk_value*100;
    
    if risk_value>100:
        risk_value=98
    
    df_routes.loc[index,'risk_score']=risk_value
    
    

C:\Users\local_jegadeesan.rameshbab12\Temp\ipykernel_35204\1105421730.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '15.944321417273013' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_routes.loc[index,'risk_score']=risk_value


In [28]:
df_routes.to_clipboard()